# Prepare annotations for image segmentation

Author: Wenxin Yang, 08/08/2024

Purpose of script: After creating image chips (i.e., slicing raster images into 256 px by 256 px), we convert the vector urban green space features (currently proxied by parks) into analysis-ready images.

Prior to running this script, we used ArcPro's Export Training Data for Deep Leanring to create those 256 px by 256 px image chips. We intend to use open source packages to replace this step in the future version.

### 1. Import necessary packages and get path to important folders

In [57]:
import os
import geopandas as gpd
import numpy as np
import rasterio as rio
from shapely.geometry import box
from rasterio import features

In [3]:
os.getcwd()

'/Users/wenxinyang/Desktop/SideProj/iguide2024/script'

In [23]:
# path to the image chips
data_folder = r'/users/wenxinyang/desktop/sideproj/iguide2024/data'
train_folder = r'/Users/wenxinyang/desktop/sideproj/iguide2024/data/training_samples'
img_folder = os.path.join(train_folder, 'images')
annot_folder = os.path.join(train_folder, 'annotations')
if os.path.exists(annot_folder)==False:
    os.mkdir(os.path.join(train_folder, 'annotations'))

In [18]:
# get all images in the img folder
imgs = [x for x in os.listdir(img_folder) if x.endswith('.tif')]

In [20]:
len(imgs)

586

### 2. [Incomplete] Check completeness of image chips

In [ ]:
# because I did not use a buffer when preparing the background image, there is a problem, some chips are incomplete
# should've added the buffer
# but for now, we could check if the image falls completely within 2020_houston_naip.tif

In [ ]:
# to be edited
def filterIncompleteChips():
    print('')

### 3. Convert urban green space features into binary images

In [81]:
# read in the feature data
prk = gpd.read_file(os.path.join(data_folder, 'park_hst.shp'))

In [78]:
# our function to create annotations
def createAnnot(folder, img_name, shp):
    # 0. read in the image
    path_img = os.path.join(folder, img_name)
    img = rio.open(path_img)
    
    
    # 1. crop park by the bounding box of the image
    # 1.1 get bounding box of the image
    bounds = img.bounds
    geom = box(*bounds)
    gdf = gpd.GeoDataFrame({"id":1,"geometry":[geom]})
    # 1.2 crop park by it
    shp_img = gdf.clip(shp).explode(index_parts = True)
    
    # 2. convert the cropped shapefile to a binary raster: 0 means non-UGS, 1 means UGS
    # 2.1 rasterize
    indv_geom = [shapes for shapes in prk.geometry]
    rasterized = features.rasterize(indv_geom,
                                    out_shape = img.shape,
                                    fill = 0,
                                    out = None,
                                    transform = img.transform,
                                    all_touched = False,
                                    default_value = 1,
                                    dtype = None)
    
    # 2.2 save raster in the annotations folder
    with rio.open(
        os.path.join(annot_folder, img_name), "w",
        driver = "GTiff",
        crs = img.crs,
        transform = img.transform,
        dtype = rio.uint8,
        count = 1,
        width = img.width,
        height = img.height) as dst:
        dst.write(rasterized, indexes = 1)
    
    print(img_name + ' done')

In [80]:
for i in imgs:
    createAnnot(img_folder, i, prk)

/opt/anaconda3/envs/py10/lib/python3.10/site-packages/geopandas/geodataframe.py:2361: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  


000000000448.tif done


/opt/anaconda3/envs/py10/lib/python3.10/site-packages/geopandas/geodataframe.py:2361: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  


000000000312.tif done


/opt/anaconda3/envs/py10/lib/python3.10/site-packages/geopandas/geodataframe.py:2361: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  


...

000000000337.tif done


/opt/anaconda3/envs/py10/lib/python3.10/site-packages/geopandas/geodataframe.py:2361: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  


000000000486.tif done


/opt/anaconda3/envs/py10/lib/python3.10/site-packages/geopandas/geodataframe.py:2361: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  


000000000492.tif done


### 4. [Incomplete] Convert .tiff image chips into png

Because many computer vision packages do not accept .tif files as inputs, we need to convert .tif files into .png files before feeding them into models. There are two relatively straightforward and achievable ways to do so. One option would be to drop the near infrared band for NAIP and, treat them like RGB pictures, and convert .tif files to .png files. The other way (credits to Alex), which applies to all multispectral inputs, would be to apply a principal component analysis and use the top three principal components as the three channels in the inputs for CV models. We intend to try these methods and also look for other packages/methods to overcome this issue. Up till now, we tried keras in Python and imageseg in R.


Our next notebook includes basic codes to train and run a CV model for image segmentation of urban green spaces, assuming we are able to create these image chips and annotations that are acceptable by the packages.